In [1]:
import os
import re
import numpy as np
import pandas as pd
import pdfplumber
from pdf2image import convert_from_path
import pytesseract


In [2]:
pd.set_option("display.max_colwidth", 200)

company_name = "delta"
os.makedirs("output", exist_ok=True)
os.makedirs(f"data/{company_name}", exist_ok=True)


input_files = {
    # "annual_report_2020": "input/10K Report 2020.pdf",
    # "annual_report_2021": "input/10K Report 2021.pdf",
    # "annual_report_2022": "input/10K Report 2022.pdf",
    # "annual_report_2023": "input/10K Report 2023.pdf",
    "annual_report_2024": "input/10K Report 2024.pdf",
    # "esg_report_2020": "input/ESG Report 2020.pdf",
    # "esg_report_2021": "input/ESG Report 2021.pdf",
    # "esg_report_2022": "input/ESG Report 2022.pdf",
    # "esg_report_2023": "input/ESG Report 2023.pdf",
    "esg_report_2024": "input/ESG Report 2024.pdf",
    "major_holders": "input/Major Holders Summary.pdf"
}

# <span style="color: aquamarine;">I. PDF Extraction and Cleaning</span>

### <span style="color: yellow">Clean Text</span>


In [3]:
def clean_text(text):
    if not text:
        return ""

    text = re.sub(r"[ \t]+", " ", text) #Replace multiple spaces/tabs with a single space
    text = re.sub(r"\n\s*\n\s*\n+", "\n\n", text) #Replace multiple blank lines into 1 blank line
    text = text.replace("\x00", "") #Remove NULL characters
    text = text.replace("\uf0b7", "•") #Replace odd Unicode bullets in PDFs to normal bullet point
    #Replace smart quotes with normal quotes
    text = text.replace("\u2019", "'") 
    text = text.replace("\u201c", '"')
    text = text.replace("\u201d", '"')

    return text.strip()


### <span style="color: yellow">Multi-Column Extraction</span>

##### <span style="color: pink">As many pdfs use a two-column layout which might extract texts in the wrong order (e.g., reading across columns instead of down), this function would split the page into left and right columns, extract text from each separately, combine them in reading order, compare with normal extraction, and finally chooose version with higher number of texts.</span>


In [4]:
def extract_text_with_columns(page):
    width = page.width
    height = page.height
    mid = width / 2

    # 2-column split
    left = page.crop((0, 0, mid, height))
    right = page.crop((mid, 0, width, height))

    left_text = left.extract_text(layout=True) or ""
    right_text = right.extract_text(layout=True) or ""
    two_col = left_text + "\n" + right_text

    # 1-column extract
    single = page.extract_text(layout=True) or ""

    return two_col if len(two_col) > len(single) else single


### <span style="color: yellow">OCR Extraction</span>

##### <span style="color: pink">As ESG report is not text-only, Optical Character Recognition is required to work with image-embded pages. This function converts a specific PDF page into an image, runs OCR, and finally cleans the text.</span>


In [5]:
def extract_text_ocr(pdf_path, page_number):
    #Store all images into a list
    images = convert_from_path(
        pdf_path,
        first_page=page_number+1,
        last_page=page_number+1
    )

    # A page may render into multiple image tiles, so after checking if more than one images are produced for the same page, using the first image is fine.
    img = images[0]
    text = pytesseract.image_to_string(img)
    return clean_text(text)


In [6]:
# with pdfplumber.open("input/ESG Report 2024.pdf") as pdf:
#     print("Number of pages:", len(pdf.pages))

In [ ]:
# #DO NOT USE: this is for testing of the previous function

# def debug_pdf_images(pdf_path, page_number):
#     images = convert_from_path(
#         pdf_path,
#         first_page=page_number+1,
#         last_page=page_number+1
#     )
    
#     print(f"Page {page_number+1} produced {len(images)} image(s):")
#     for i, img in enumerate(images):
#         print(f" - Image {i}: size={img.size}")


# def scan_pdf_for_image_tiles(pdf_path):
#     from pdf2image import convert_from_path
#     import pdfplumber

#     with pdfplumber.open(pdf_path) as pdf:
#         n_pages = len(pdf.pages)

#     for page_num in range(n_pages):
#         images = convert_from_path(
#             pdf_path,
#             first_page=page_num+1,
#             last_page=page_num+1
#         )
#         print(f"Page {page_num+1} produced {len(images)} image(s)")

In [8]:
# debug_pdf_images("input/ESG Report 2024.pdf", 0)   
# debug_pdf_images("input/ESG Report 2024.pdf", 1)  


In [9]:

# scan_pdf_for_image_tiles("input/ESG Report 2024.pdf")


### <span style="color: yellow">Hybrid Page Extraction</span>

##### <span style="color: pink">ESG documents → OCR-first</span>
##### <span style="color: pink">Annual reports → pdfplumber-first</span>


In [10]:
def is_esg_file(filename):
    name = os.path.basename(filename).lower()
    return "esg" in filename.lower()

In [43]:
def extract_page_hybrid(pdf_path, page, page_number, is_esg, ocr_threshold=60):
    if is_esg:
        # Try text extraction with columns
        pdf_text = extract_text_with_columns(page)
        pdf_text_clean = clean_text(pdf_text)
        pdf_len = len(pdf_text_clean)

        if pdf_len > ocr_threshold:
            print(f" → Page {page_number+1}: ESG: pdfplumber ({pdf_len} chars)")
            return pdf_text_clean

        # Otherwise OCR
        print(f" → Page {page_number+1}: ESG: OCR fallback ({pdf_len} chars)")
        return extract_text_ocr(pdf_path, page_number)

    else:
        # Use single-column extraction to avoid chopping characters
        pdf_text = page.extract_text(layout=True) or ""
        pdf_text_clean = clean_text(pdf_text)
        pdf_len = len(pdf_text_clean)

        if pdf_len == 0:
            # Only then try OCR as a last resort
            print(f" → Page {page_number+1}: 10-K: EMPTY → OCR fallback")
            return extract_text_ocr(pdf_path, page_number)

        print(f" → Page {page_number+1}: 10-K: pdfplumber only ({pdf_len} chars)")
        return pdf_text_clean


In [ ]:
#STILL TESTING - DO NOT USE
def extract_page_hybrid(pdf_path, page, page_number, is_esg, ocr_threshold=60):
    if is_esg:
        ocr_text = extract_text_ocr(pdf_path, page_number)
        ocr_len = len(ocr_text.strip())

        if ocr_len >= ocr_threshold:
            return ocr_text

        # Fallback to pdfplumber if OCR is too weak
        pdf_text = extract_text_with_columns(page)
        pdf_len = len(pdf_text.strip())

        return clean_text(pdf_text)

    # pdf_text = extract_text_with_columns(page)
    # pdf_len = len(pdf_text.strip())

    pdf_text = page.extract_text(layout=True) or ""
    pdf_text_clean = clean_text(pdf_text)
    pdf_len = len(pdf_text_clean)

    if pdf_len == 0:
        # Only then try OCR as a last resort
        print(f" → Page {page_number+1}: 10-K: EMPTY → OCR fallback")
        return extract_text_ocr(pdf_path, page_number)
    
    # if pdf_len >= ocr_threshold:
    #     return clean_text(pdf_text)

    # print(f" → Page {page_number+1}: 10-K: pdfplumber only ({pdf_len} chars)")
    return extract_text_ocr(pdf_path, page_number)



### <span style="color: yellow">Full Hybrid Extraction</span>


In [12]:
def extract_text_hybrid(pdf_path):
    print(f"\nExtracting: {pdf_path}")
    full_text = ""

    with pdfplumber.open(pdf_path) as pdf:
        n_pages = len(pdf.pages)
        print(f"PDF has {n_pages} pages")

        for i, page in enumerate(pdf.pages):
            page_text = extract_page_hybrid(pdf_path, page, i, is_esg=is_esg_file(pdf_path))
            full_text += page_text + f"\n\n--- PAGE {i+1} END ---\n\n"

    return full_text.strip()


# <span style="color: aquamarine;">II. Text Chunking</span>

### <span style="color: yellow">Chunk Text</span>

#### <span style="color: pink">This step is to split text into chunks of 600. Parameters include text (text to chunk), chunk_size (target words per chunk), and overlap (words to overlap between chunks).</span>

In [13]:
def chunk_text(text, chunk_size=600, overlap=100):
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunk_words = words[start:end]
        chunk = ' '.join(chunk_words)
        chunks.append(chunk)
        start += max(chunk_size - overlap, 1)

    return chunks


### <span style="color: yellow">Create Document Store</span>


In [14]:
def create_document_store(cleaned_texts, company_name, chunk_size=600, overlap=100):
    
    all_chunks = []
    chunk_counter = 1
    
    print("\nCreating document store...")
    print("="*60)
    
    for source_file, text in cleaned_texts.items():
        print(f"\nProcessing: {source_file}")
        
        # Chunk the text
        chunks = chunk_text(text, chunk_size=chunk_size, overlap=overlap)
        print(f"  Created {len(chunks)} chunks")
        
        # Add each chunk to the list
        for i, chunk in enumerate(chunks):
            chunk_data = {
                'chunk_id': f"chunk_{chunk_counter:03d}",
                'company': company_name,
                'source_file': source_file,
                'chunk_text': chunk
            }
            all_chunks.append(chunk_data)
            chunk_counter += 1
    
    df = pd.DataFrame(all_chunks)
    
    print("\n" + "="*60)
    print(f"✓ Document store created!")
    print(f"  Total chunks: {len(df)}")
    print(f"  Total documents: {len(cleaned_texts)}")
    print("="*60)
    
    return df

# <span style="color: aquamarine;">III. Run Extraction → Cleaning → Chunking</span>

In [15]:
cleaned_texts = {}

for name, path in input_files.items():
    print("\n" + "="*60)
    print(f"Processing {name}")
    print("="*60)

    raw = extract_text_hybrid(path)
    cleaned = clean_text(raw)

    cleaned_texts[name] = cleaned

    outpath = f"output/{name}_clean.txt"
    with open(outpath, "w", encoding="utf-8") as f:
        f.write(cleaned)
    print(f"✓ Saved cleaned text: {outpath}")



Processing annual_report_2024

Extracting: input/10K Report 2024.pdf
PDF has 104 pages
 → Page 1: 10-K: pdfplumber only (4306 chars)
 → Page 2: 10-K: pdfplumber only (1533 chars)
 → Page 3: 10-K: pdfplumber only (265 chars)
 → Page 4: 10-K: pdfplumber only (1225 chars)
 → Page 5: 10-K: pdfplumber only (3530 chars)
 → Page 6: 10-K: pdfplumber only (4745 chars)
 → Page 7: 10-K: pdfplumber only (4184 chars)
 → Page 8: 10-K: pdfplumber only (4378 chars)
 → Page 9: 10-K: pdfplumber only (3478 chars)
 → Page 10: 10-K: pdfplumber only (4935 chars)
 → Page 11: 10-K: pdfplumber only (4039 chars)
 → Page 12: 10-K: pdfplumber only (4545 chars)
 → Page 13: 10-K: pdfplumber only (4123 chars)
 → Page 14: 10-K: pdfplumber only (5429 chars)
 → Page 15: 10-K: pdfplumber only (5214 chars)
 → Page 16: 10-K: pdfplumber only (6159 chars)
 → Page 17: 10-K: pdfplumber only (4673 chars)
 → Page 18: 10-K: pdfplumber only (5154 chars)
 → Page 19: 10-K: pdfplumber only (717 chars)
 → Page 20: 10-K: pdfplumber o

In [17]:
document_store = create_document_store(
    cleaned_texts,
    company_name=company_name,
    chunk_size=600,
    overlap=100
)

document_store["word_count"] = document_store["chunk_text"].str.split().str.len()

csv_path = f"data/{company_name}/chunks.csv"
document_store.to_csv(csv_path, index=False)
print(f"\n✓ Saved chunks → {csv_path}")



Creating document store...

Processing: annual_report_2024
  Created 112 chunks

Processing: esg_report_2024
  Created 46 chunks

Processing: major_holders
  Created 1 chunks

✓ Document store created!
  Total chunks: 159
  Total documents: 3

✓ Saved chunks → data/delta/chunks.csv


In [19]:
document_store.head(10)

,chunk_id,company,source_file,chunk_text,word_count
0,chunk_001,delta,annual_report_2024,"UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM 10-K ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 iS} For the fiscal year ended...",600
1,chunk_002,delta,annual_report_2024,statements of the registrant included in the filing reflect the correction of an error to previously issued financial statements 0 Indicate by check mark whether any of those error corrections are...,600
2,chunk_003,delta,annual_report_2024,"actual results to differ materially from historical experience or our present expectations. Known material risk factors applicable to Delta are described in ""Risk Factors Relating to Delta"" and ""R...",600
3,chunk_004,delta,annual_report_2024,"six continents. Our domestic network is centered around core hubs in Atlanta, Detroit, Minneapolis-St. Paul and Salt Lake City. Core hubs have strong local passenger share, a high penetration of c...",600
4,chunk_005,delta,annual_report_2024,"consumer brand, including: + Recognized as the 2024 Airline of the Year by aviation publication Air Transport World. + Named Best U.S. Airline and topped five categories in the Forbes Travel Guide...",600
5,chunk_006,delta,annual_report_2024,"mileage credits (""miles"") when traveling on Delta, Delta Connection and our partner airlines. Miles may also be earned by using certain services offered by program partners, such as credit card, r...",600
6,chunk_007,delta,annual_report_2024,Consolidated Financial Statements for additional information about our equity investments. Each of our joint venture or cooperation arrangements provides for joint commercial cooperation with the ...,600
7,chunk_008,delta,annual_report_2024,"the other member airlines, providing opportunities to increase connecting traffic while offering enhanced customer service through reciprocal codesharing and loyalty program participation, airport...",600
8,chunk_009,delta,annual_report_2024,"from around the world. With agreements to service multiple next-generation aircraft engines, Delta TechOps is positioned as a leading global service provider for state-of-the-art, more sustainable...",600
9,chunk_010,delta,annual_report_2024,"work with airport partners to drive fuel savings by limiting the use of aircraft Auxiliary Power Units (""APUs"") during ground operations. In 2024, we installed 30 new preconditioned air units at o...",600
